In [ ]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [ ]:
x = np.loadtxt('test_diode_igbt_capacitor_load_(12_parameter).txt', delimiter=',')
x = np.expand_dims(x,axis=2)
train_labels=(x[0:7650,0])-1
train_samples=(x[0:7650,1:2401])
print(train_samples.shape)

In [ ]:
labels_names = ['Healthy','OC DIODE','SC DIODE', 'OC IGBT', 'SC IGBT', 'OC CAPACITOR','OC LOAD','SC LOAD']

In [ ]:
train_labels, train_samples = shuffle(train_labels, train_samples)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.model_selection import train_test_split

In [ ]:
model = keras.Sequential([
 keras.layers.Reshape((600,4), input_shape=(2400,1)),
 keras.layers.Conv1D(filters=300, kernel_size=10, 
activation='relu',padding='valid', input_shape=(600, 4)),
 keras.layers.MaxPooling1D(),
 keras.layers.Conv1D(filters=200, kernel_size=5, 
activation='relu',padding='valid'),
 keras.layers.MaxPooling1D(),
 keras.layers.Dropout(0.5),
 keras.layers.Conv1D(filters=200, kernel_size=3, 
activation='relu',padding='valid'),
 keras.layers.MaxPooling1D(),
 keras.layers.Dropout(0.5),
 keras.layers.Flatten(),
 keras.layers.Dense(200, activation="relu"),
 keras.layers.Dense(100, activation="relu"),
  keras.layers.Dense(50, activation="relu"),
 keras.layers.Dense(8, activation="softmax")])


In [ ]:
model.summary()

In [ ]:
data_train, data_test, labels_train, labels_test = train_test_split(train_samples, train_labels, test_size=0.20, random_state=42)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_samples, train_labels, epochs=10)

In [ ]:
test_loss, test_acc = model.evaluate(data_test, labels_test)
nofault_loss, nofault_acc = model.evaluate(train_samples[0:224,0:2400],train_labels[0:224])
OCD_loss, OCD_acc = model.evaluate(train_samples[225:1124,0:2400],train_labels[225:1124])
SCD_loss, SCD_acc = model.evaluate(train_samples[1125:2024,0:2400],train_labels[1125:2024])
OCI_loss, OCI_acc = model.evaluate(train_samples[2025:3374,0:2400],train_labels[2025:3374])
SCI_loss, SCI_acc = model.evaluate(train_samples[3375:4724,0:2400],train_labels[3375:4724])
OCC_loss, OCC_acc = model.evaluate(train_samples[4725:5849,0:2400],train_labels[4725:5849])
OCL_loss, OCL_acc = model.evaluate(train_samples[5850:6749,0:2400],train_labels[5850:6749])
SCL_loss, SCL_acc = model.evaluate(train_samples[6750:7649,0:2400],train_labels[6750:7649])

print('\nTest accuracy:', test_acc*100, '%')
prediction=model.predict(data_test)

for i in range(20):
 print("Prediction: ", labels_names[np.argmax(prediction[i])], " Actual: ", labels_names[int(labels_test[i])])

print('\nHealthy accuracy:', nofault_acc*100, '%')
print('\nOpen Circuit Diode accuracy:', OCD_acc*100, '%')
print('\nShort Circuit Diode accuracy:', SCD_acc*100, '%')
print('\nOpen Circuit IGBT accuracy:', OCI_acc*100, '%')
print('\nShort Circuit IGBT accuracy:', SCI_acc*100, '%')
print('\nOpen Circuit Capacitor accuracy:', OCC_acc*100, '%')
print('\nOpen Circuit Load accuracy:', OCL_acc*100, '%')
print('\nShort Circuit Load accuracy:', SCL_acc*100, '%')

